In [1]:
from bs4 import BeautifulSoup # to parse external data
import yfinance as yf
import pandas as pd # to read CSV files
import requests # to get data
import spacy #to extraxt entities
import streamlit as st

In [2]:
st.title('Stocks overview')

2021-12-22 21:28:03.448 
  command:

    streamlit run /Users/Martin/opt/anaconda3/lib/python3.8/site-packages/ipykernel_launcher.py [ARGUMENTS]


DeltaGenerator(_root_container=0, _provided_cursor=None, _parent=None, _block_type=None, _form_data=None)

In [3]:
nlp = spacy.load('en_core_web_sm')

In [4]:
def extract_text_from_rss(rss_link):
    """ Extracts the headlines from the links """
    headings = []
    r1 = requests.get('https://www.cnbc.com/id/15839135/device/rss/rss.html?fbclid=IwAR2o0zeWtmgEwZob45_F6e02pkTVo9uBGL0VI1GQv8mPyScEFY-hn9t089Y')
    r2 = requests.get(rss_link)
    soup1 = BeautifulSoup(r1.content, features='xml')
    soup2 = BeautifulSoup(r2.content, features='xml')
    headings1 = soup1.findAll('title')
    headings2 = soup2.findAll('title')
    headings = headings1 + headings2
    return headings

In [5]:
stocks_df = pd.read_csv('./Nifty500.csv')  ## https://www1.nseindia.com/products/content/equities/indices/nifty_500.htm
stocks_df.head()

,Company Name,Industry,Symbol,Series,ISIN Code
0,3M India Ltd.,CONSUMER GOODS,3MINDIA,EQ,INE470A01017
1,ABB India Ltd.,INDUSTRIAL MANUFACTURING,ABB,EQ,INE117A01022
2,ACC Ltd.,CEMENT & CEMENT PRODUCTS,ACC,EQ,INE012A01025
3,AIA Engineering Ltd.,INDUSTRIAL MANUFACTURING,AIAENG,EQ,INE212H01026
4,APL Apollo Tubes Ltd.,METALS,APLAPOLLO,EQ,INE702C01027


In [6]:
stock_info = yf.Ticker('3MINDIA.NS') ## without NS there is no info
stock_info.info

2021-12-22 21:28:05.667 NumExpr defaulting to 8 threads.


{'zip': '560001',
 'sector': 'Industrials',
 'fullTimeEmployees': 1146,
 'longBusinessSummary': '3M India Limited engages in the manufacture and trade of various products for health care, manufacturing, automotive, safety, electronics, energy, commercial solutions, transportation, and design and construction industries in India. The company operates through four segments: Safety and Industrial, Transportation and Electronics, Health Care, and Consumer. The Safety and Industrial segment offers vinyl, polyester, foil, and specialty industrial tapes and adhesives, such as scotch masking tapes, scotch filament and packaging tapes, functional and decorative graphics, abrasion-resistant films, masking tapes, and other specialty materials. This segment serves industrial, electrical, and safety markets. The Health Care segment provides medical and surgical supplies; medical devices; skin and wound care, as well as infection prevention products and solutions; drug delivery systems; dental and o

In [7]:
stocksSP500 = pd.read_csv('./SP500.csv') ## https://github.com/datasets/s-and-p-500-companies
stocksSP500.head() 

,Symbol,Name,Sector
0,MMM,3M,Industrials
1,AOS,A. O. Smith,Industrials
2,ABT,Abbott Laboratories,Health Care
3,ABBV,AbbVie,Health Care
4,ABMD,Abiomed,Health Care


In [8]:
stock_info_SP500 = yf.Ticker('MMM')
stock_info_SP500.info

{'zip': '55144-1000',
 'sector': 'Industrials',
 'fullTimeEmployees': 95000,
 'longBusinessSummary': '3M Company develops, manufactures, and markets various products worldwide. It operates through four business segments: Safety and Industrial, Transportation and Electronics, Health Care, and Consumer. The Safety and Industrial segment offers personal safety products, industrial adhesives and tapes, abrasives, closure and masking systems, electrical markets, automotive aftermarket, and roofing granules to industrial, electrical, and safety markets. The Transportation and Electronics provides electronics, such as display materials and systems, electronic materials solutions; automotive and aerospace, and commercial solutions; advanced materials; and transportation safety products to transportation and electronic original equipment manufacturer customers. The Health Care segment offers medical and surgical supplies, skin health and infection prevention products, oral care, separation and 

In [9]:
def generate_stock_info(headings):
    """ Goes over each heading to found out the entitites and link it with Nifty 500 companies data
    and Extract the market data using yahoo finance ticker function 
    
    Return: data frame containing all the buzzing stocks and their stats
    """
    Stock_info_dict = {
        'Org': [],
        'Symbol': [],
        'currentPrice': [],
        'dayHigh': [],
        'dayLow': [],
        'forwardPE': [],
        'dividendYield': []
    }
    
    stocks_df = pd.read_csv('./Nifty500.csv')
    for title in headings:
        doc = nlp(title.text)
        for ent in doc.ents:
            try:
                if stocks_df['Company Name'].str.contains(ent.text).sum():
                    symbol = stocks_df[stocks_df['Company Name'].str.contains(ent.text)]['Symbol'].values[0]
                    org_name = stocks_df[stocks_df['Company Name'].str.contains(ent.text)]['Company Name'].values[0]
                        #  sending yfinance the symbol for stock info
                    stock_info = yf.Ticker(symbol+".NS").info
                    print(symbol)
                     
                    stock_info_dict['Org'].append(org_name)
                    stock_info_dict['Symbol'].append(symbol)
                        
                    stock_info_dict['currentPrice'].append(stock_info['currentPrice'])
                    stock_info_dict['dayHigh'].append(stock_info['dayHigh'])
                    stock_info_dict['dayLow'].append(stock_info['dayLow'])
                    stock_info_dict['forwardPE'].append(stock_info['forwardPE'])
                    stock_info_dict['dividendYield'].append(stock_info['dividendYield'])
                else:
                    pass
            except:
                pass

    output_df = pd.DataFrame(Stock_info_dict)
    return output_df


In [21]:
# add an input field to pass the RSS link
user_input = st.text_input('Add your RSS link here', 'https://www.cnbc.com/id/15839135/device/rss/rss.html?fbclid=IwAR2o0zeWtmgEwZob45_F6e02pkTVo9uBGL0VI1GQv8mPyScEFY-hn9t089Y')
user_input

'https://www.cnbc.com/id/15839135/device/rss/rss.html?fbclid=IwAR2o0zeWtmgEwZob45_F6e02pkTVo9uBGL0VI1GQv8mPyScEFY-hn9t089Y'

In [22]:
# get the financial headlines
fin_headings = extract_text_from_rss(user_input)

In [23]:
## output the financial info
output_df = generate_stock_info(fin_headings)
output_df.drop_duplicates(inplace = True)
st.dataframe(output_df)

DeltaGenerator(_root_container=0, _provided_cursor=None, _parent=None, _block_type=None, _form_data=None)

In [24]:
## Display the headlines
with st.expander("Expand for financial stocks news!"):
    for heading in fin_headings:
        st.markdown("* " + heading.text)